In [1]:
from bibleabbr import *

In [2]:
import numpy as np
import shorttext
import sqlite3

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Scripture Loaded

In [3]:
dbconn = sqlite3.connect('ESVCrossway.sqlite')

In [4]:
from functools import reduce
from nltk import sent_tokenize

def retrieve_docs_as_biblechapters(dbconn):
    cursor = dbconn.cursor()
    for book in wholebible_book_iterator():
        for chap in range(1, numchaps[book]+1):
            doc_label = book+'_'+str(chap)
            result = cursor.execute('select scripture from bible where book is "'+book+'" and chapter='+str(chap))
            doc_text = reduce(lambda s1, s2: ' '.join([s1, s2]), [texttuple[0] for texttuple in result])
            yield doc_label, doc_text
    cursor.close()
    

In [5]:
from collections import defaultdict

classdict = defaultdict(lambda : [])
for bible_chap, verses in retrieve_docs_as_biblechapters(dbconn):
    for verse in sent_tokenize(verses):
        classdict[bible_chap] += [verse]

# C-LSTM

In [7]:
wmodel = shorttext.utils.load_word2vec_model('wordembedD/wiki-news-300d-1M.vec', binary=False)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
kmodel = shorttext.classifiers.CLSTMWordEmbed(len(classdict.keys()), vecsize=wmodel.vector_size)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wmodel)

In [10]:
classifier.train(classdict, kmodel)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
35885/35885 [==============================] - 198s 6ms/step - loss: 6.0175
Epoch 2/10
35885/35885 [==============================] - 194s 5ms/step - loss: 4.3289
Epoch 3/10
35885/35885 [==============================] - 195s 5ms/step - loss: 3.1450
Epoch 4/10
35885/35885 [==============================] - 192s 5ms/step - loss: 2.0375
Epoch 5/10
35885/35885 [==============================] - 192s 5ms/step - loss: 1.1264
Epoch 6/10
35885/35885 [==============================] - 191s 5ms/step - loss: 0.6314
Epoch 7/10
35885/35885 [==============================] - 190s 5ms/step - loss: 0.4355
Epoch 8/10
35885/35885 [==============================] - 190s 5ms/step - loss: 0.3670
Epoch 9/10
35885/35885 [==============================] - 191s 5ms/step - loss: 0.3299
Epoch 10/10
35885/35885 [==============================] - 191s 5ms/step - loss: 0.2964


In [12]:
classifier.save_compact_model('biblechap_CLSTM_ft.pkl')

# Prediction

In [4]:
import shorttext

wmodel = shorttext.utils.load_word2vec_model('wordembedD/wiki-news-300d-1M.vec', binary=False)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
classifier = shorttext.smartload_compact_model('biblechap_CLSTM_ft.pkl', wmodel)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
sorted(classifier.score('justice').items(), key=lambda item: item[1], reverse=True)[:10]

[('ps_9', 0.11736146),
 ('pr_16', 0.03148068),
 ('is_29', 0.028963896),
 ('pr_21', 0.027923644),
 ('ps_7', 0.026551569),
 ('ho_9', 0.023678044),
 ('ps_58', 0.018452283),
 ('ps_150', 0.017172663),
 ('job_34', 0.015781064),
 ('ps_33', 0.01547823)]